In [2]:
import pandas as pd
import os
from sklearn.impute import SimpleImputer
from sklearn import tree
from sklearn import model_selection

In [4]:
#change the working directory
os.chdir("D:\harish")


In [6]:
titanic_train=pd.read_csv("train.csv")
print(titanic_train.shape)

(891, 12)


In [7]:
titanic_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [8]:
titanic_test=pd.read_csv("test.csv")

In [9]:
titanic_test.shape

(418, 11)

In [10]:
titanic_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [11]:
titanic_test.Survived=None

In [17]:
titanic_test['Survived']=None

In [18]:
titanic_test['Survived']

0      None
1      None
2      None
3      None
4      None
       ... 
413    None
414    None
415    None
416    None
417    None
Name: Survived, Length: 418, dtype: object

In [19]:
#lets concatinate both train and test
#we have same number of rows and cols
#by concatinating train and test our job will be easy
titanic=pd.concat([titanic_train,titanic_test])


In [20]:
titanic.shape

(1309, 12)

In [21]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    object 
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(2), int64(4), object(6)
memory usage: 132.9+ KB


In [28]:
# Extract and create title column from name
def extract_title(name):
    return name.split(',')[1].split(',')[0].strip()
titanic['Title']=titanic['Name'].map(extract_title)

In [54]:
titanic.Name.iloc[1:2,][1]

'Cumings, Mrs. John Bradley (Florence Briggs Thayer)'

In [52]:
titanic.Title.iloc[1:2,][1]


'Mrs. John Bradley (Florence Briggs Thayer)'

In [55]:
#imputation work for missing data with default values
# Age column has missing data
# Fare column has no missing data
mean_imputer=SimpleImputer()

In [64]:
mean_imputer.fit(titanic_train[['Age','Fare']])# By default parameter is mean

SimpleImputer()

In [65]:
titanic[['Age','Fare']]=mean_imputer.transform(titanic[['Age','Fare']])

In [66]:
# no null values in age column
titanic['Age'].info()

<class 'pandas.core.series.Series'>
Int64Index: 1309 entries, 0 to 417
Series name: Age
Non-Null Count  Dtype  
--------------  -----  
1309 non-null   float64
dtypes: float64(1)
memory usage: 20.5 KB


In [67]:
#no null values in Fare column
titanic['Fare'].info()

<class 'pandas.core.series.Series'>
Int64Index: 1309 entries, 0 to 417
Series name: Fare
Non-Null Count  Dtype  
--------------  -----  
1309 non-null   float64
dtypes: float64(1)
memory usage: 20.5 KB


In [69]:
'''creating categorical age column from age
its always a good practice to create functions 
so that the same can be applies on test data as well  '''
def convert_age(age):
    if(age>=0 and age<=10):
        return 'Child'
    elif(age<=25):
        return 'Young'
    elif(age<=50):
        return 'Middle'
    else:
        return 'Old'
#convert numerical age column to categorical age_cat column
titanic['Age_cat']=titanic['Age'].map(convert_age)

In [73]:
#create a new familysize by combining sibsp and parch
titanic['FamilySize']=titanic['SibSp']+titanic['Parch']+1
def convert_familysize(size):
    if(size==1):
        return 'Single'
    elif(size<=3):
        return 'Small'
    elif(size<=6):
        return 'Medium'
    else:
        return 'Large'
# convert numerical familysize column to categorical familysize_cat column
titanic['FamilySize_Cat']=titanic['FamilySize'].map(convert_familysize)

In [122]:
titanic['FamilySize_Cat']

0       Small
1       Small
2      Single
3       Small
4      Single
        ...  
413    Single
414    Single
415    Single
416    Single
417     Small
Name: FamilySize_Cat, Length: 1309, dtype: object

In [125]:
#Now we got 3 new columns, Title, Age_Cat, FamilySize_Cat
#convert categorical columns to one-hot encoded columns including, 
#newly created 3 categorical columns
titanic1=pd.get_dummies(titanic,columns=['Sex','Pclass','Age_cat','Title','FamilySize_Cat','Embarked','FamilySize_Cat'])
print(titanic1.shape)
titanic1.info()

(1309, 1173)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Columns: 1173 entries, PassengerId to FamilySize_Cat_Small
dtypes: float64(2), int64(4), object(4), uint8(1163)
memory usage: 1.6+ MB


In [128]:
# Drop unwanted columns and create new set called titanic2
titanic2=titanic1.drop(['PassengerId','Name','Age','Ticket','Cabin','Survived'],axis=1,inplace=False)

In [129]:
titanic2.shape

(1309, 1167)

In [130]:
titanic2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Columns: 1167 entries, SibSp to FamilySize_Cat_Small
dtypes: float64(1), int64(3), uint8(1163)
memory usage: 1.5 MB


In [189]:
# Splitting train and test data
x_train=titanic2[0:891]

In [190]:
x_train.shape

(891, 1167)

In [191]:
y_train=titanic_train['Survived']

In [192]:
#Let's build the model
#If we don't use random_state parameter, system can pick different values each time and we may get slight difference in accuracy each time you run.
tree_estimator=tree.DecisionTreeClassifier()

In [193]:
tree_estimator

DecisionTreeClassifier()

In [194]:
# Add parameters for tuning
dt_grid={'max_depth':[10,12],'min_samples_split':[2,4,6,7,8],'criterion':['gini','entropy']}

In [195]:
param_grid=model_selection.GridSearchCV(tree_estimator,dt_grid,cv=9)

In [196]:
param_grid.fit(x_train,y_train)

GridSearchCV(cv=9, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 12],
                         'min_samples_split': [2, 4, 6, 7, 8]})

In [197]:
print(param_grid.best_score_) #Best score
print(param_grid.best_params_)
print(param_grid.score(x_train, y_train)) #Train score  #Evalution of tree

0.8294051627384962
{'criterion': 'entropy', 'max_depth': 12, 'min_samples_split': 4}
0.9438832772166106


In [227]:
# Now lets predict the test data
x_test=titanic2[891:]

In [199]:
x_test.shape

(418, 1167)

In [225]:
titanic2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Columns: 1167 entries, SibSp to FamilySize_Cat_Small
dtypes: float64(1), int64(3), uint8(1163)
memory usage: 1.5 MB


In [200]:
x_test.head()

,SibSp,Parch,Fare,FamilySize,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Age_cat_Child,...,FamilySize_Cat_Medium,FamilySize_Cat_Single,FamilySize_Cat_Small,Embarked_C,Embarked_Q,Embarked_S,FamilySize_Cat_Large,FamilySize_Cat_Medium,FamilySize_Cat_Single,FamilySize_Cat_Small
0,0,0,7.8292,1,0,1,0,0,1,0,...,0,1,0,0,1,0,0,0,1,0
1,1,0,7.0000,2,1,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,1
2,0,0,9.6875,1,0,1,0,1,0,0,...,0,1,0,0,1,0,0,0,1,0
3,0,0,8.6625,1,0,1,0,0,1,0,...,0,1,0,0,0,1,0,0,1,0
4,1,1,12.2875,3,1,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,1


In [201]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Columns: 1167 entries, SibSp to FamilySize_Cat_Small
dtypes: float64(1), int64(3), uint8(1163)
memory usage: 491.1 KB


In [229]:
print('X_trian shape',x_train.shape)
print("X_test shape",x_test.shape)

X_trian shape (891, 1167)
X_test shape (418, 1167)


In [207]:
titanic_test['survived']=param_grid.predict(x_test)

In [208]:
titanic_test['survived']

0      0
1      0
2      0
3      0
4      1
      ..
413    0
414    1
415    0
416    0
417    1
Name: survived, Length: 418, dtype: int64

In [210]:
#writing passengerid and survived to csv file called Attempt for prediction
titanic_test.to_csv('Attempt.csv',columns=['PassengerId','survived'],index=False)